# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing https://en.wikipedia.org/wiki/Laplace_smoothing

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
from scipy import stats
import mpmath
import matplotlib.pyplot as plt


In [2]:
def limpa(d):
    lista1 = [",", ".", "?", "@", ":", ";", "!", "'", '"', "“", "”", "#", "(", ")", "…", "/", "+", "-", "$"]
     
    k = 0        
    for n in lista1:
        d = d.replace(lista1[k], "")
        k += 1

    d = d.split()
    for p in d:
        if "https" in p:
            d.remove(p)
    return d

def limpa_cada_linha(PD, coluna):
    j = 0
    for i in PD[coluna]:
        PD[coluna][j] = limpa(PD.loc[j, coluna])
        j += 1
        
        
def retorna_D(dic1, dic2, opcoes, PD, coluna1, coluna2):
    limpa_cada_linha(PD, coluna1)
    a = 0
    for l in range(len(PD[coluna1])):
        for op in opcoes:
            if PD.loc[a, coluna2] == op:
                pedaco_D(dic1, PD.loc[a, coluna1], op)
        
        outro_pedaco_D(dic2, PD.loc[a, coluna1])
        a += 1
    return dic1, dic2

def pedaco_D(dici, lista, cat):
    for p in lista:
        if cat in dici:
            if p in dici[cat]:
                dici[cat][p] += 1
            else:
                dici[cat][p] = 1
        else:
            dici[cat] = {}
            dici[cat][p] = 1
    
def outro_pedaco_D(dici2, lista2):
    for p in lista2:
        if p in dici2:
            dici2[p] += 1
        else:
            dici2[p] = 1
            
def laplace_smothing(dict2, dict1, laplace_n):

    for p in dict1:
        dict1[p] += laplace_n
    
    
    for op in dict2:

        for p in dict2[op]:
            dict2[op][p] += laplace_n
        
        
    return  dict2, dict1

def soma_dicts(dicio1, dicio2):
    soma1 = {}
    for op in dicio1:
        soma = 0
        for p in dicio1[op]:
            soma += dicio1[op][p]
        soma1[op] = soma
    soma2 = 0    
    for p in dicio2:
        soma2 += dicio2[p]
        
    return soma1, soma2
    
def porcentagem(dicio1, dicio2, somas1, somas2):
    dicionario1 = {}
    dicionario2 = {}
    
    for op in dicio1:
        dicionario1[op] = {}
        for p in dicio1[op]:
            dicionario1[op][p] = dicio1[op][p] / somas1[op]
            
    for p in dicio2:
        dicionario2[p] = dicio2[p] / somas2
        
    return dicionario1, dicionario2

def proporcoes(PD, coluna, lista):
    opcoes = {}
    for o in lista:
        opcoes[o] = 0
        for op in PD[coluna]:
            if op == o:
                opcoes[o] += 1
                
    for d in opcoes:
        opcoes[d] = opcoes[d] / len(PD[coluna])

    return opcoes

def finaliza(porcentagem_palavras_geral, porcentagem_palavras_dado_opcao, porcentagem_da_opcao, soma_geral, soma_opcoes, PD, coluna, lista_opcoes, nomes_coluna):
    for op in lista_opcoes:
        lista = []
        for frase in PD[coluna]:
            prob_frase = mpmath.mpf(1)
            prob_frase_dado_op = mpmath.mpf(1)
            for palavra in frase:
                if palavra in porcentagem_palavras_geral:
                    prob_frase *= porcentagem_palavras_geral[palavra]
                else:
                    prob_frase *= 1/soma_geral
                if palavra in porcentagem_palavras_dado_opcao[op]:
                    prob_frase_dado_op *= porcentagem_palavras_dado_opcao[op][palavra]
                else:
                    prob_frase_dado_op *= 1/soma_opcoes[op]
            lista.append((prob_frase_dado_op * porcentagem_da_opcao[op]) / prob_frase)
        nome = nomes_coluna[op]
        PD[nome] = pd.Series((lista), index = PD.index)
        
        
    

In [3]:
#Leitura dos arquivos em Excel
TR = pd.read_excel('tweets_TESLA_201809101218.xlsx', sheet_name = 0)
TS = pd.read_excel('tweets_TESLA_201809101218.xlsx', sheet_name = 1)


In [4]:
limpa_cada_linha(TS, "Teste")
TP = TS

C:\Users\gzkaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
p_op = proporcoes(TR, "Classificação", [0, 1])

In [6]:
opcoes_d, geral_d = retorna_D({}, {}, [0, 1], TR, "Treinamento", "Classificação")

C:\Users\gzkaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
soma_opcoes, soma_geral = soma_dicts(opcoes_d, geral_d)

In [8]:
soma_opcoes

{0: 1389, 1: 4687}

In [9]:
soma_geral

6076

In [10]:
opcoes_d2, geral_d2 = laplace_smothing(opcoes_d, geral_d, 1)

In [11]:
soma_opcoes, soma_geral = soma_dicts(opcoes_d2, geral_d2)

In [12]:
soma_opcoes

{0: 2006, 1: 6054}

In [13]:
soma_geral

7769

In [14]:
por_palavra_op, por_palavra_g = porcentagem(opcoes_d2, geral_d2, soma_opcoes, soma_geral)

In [15]:
finaliza(por_palavra_g, por_palavra_op, p_op, soma_geral, soma_opcoes, TP, "Teste", [0, 1], ["P N", "P R"])


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [23]:
a = 1/soma_geral
b = 1/soma_opcoes[0]
c = p_op[0]
print((b*c)/a)

0.8907627118644067


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
